<a href="https://colab.research.google.com/github/Rohit-Roby/Project/blob/main/04_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello world")

In [ ]:
!pip install --upgrade objaverse


In [ ]:
import objaverse as obx
obx.__version__

In [ ]:
uids = obx.load_uids()
len(uids), type(uids)

In [ ]:
uids[:10]

In [ ]:
annotations = obx.load_annotations(uids)

In [ ]:
place_travel_uids = [uid for uid, annotation in annotations.items() if (len(annotation["tags"]) > 0  and annotation["tags"][0]['name'] == 'places-travel')]
place_travel_uids[:10]

In [ ]:
annotations.items()

In [ ]:
if len(annotations[uids[1]]['tags']) > 0 : print(5)

In [ ]:
import multiprocessing
processes = multiprocessing.cpu_count()
processes

In [ ]:
import random
random.seed(40)
# uids = obx.load_uids()
random_object_uids = random.sample(uids, 50)
random_object_uids

In [ ]:
objects= obx.load_objects(
    uids = random_object_uids,
    download_processes = processes
)

In [ ]:
!pip install trimesh --quiet

In [ ]:
lvis_annotations = obx.load_lvis_annotations()
lvis_annotations

use pandas and serialize and sort the descriptions of the 3d model like face count, vertex_count texture count and size, tags, description


In [6]:
!pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 36.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
    --------------------------------------- 4.6/198.6 MB 147.3 MB/s eta 0:00:02
   - ------------------------------------- 10.1/198.6 MB 129.6 MB/s eta 0:00:02
   --- ----------------------------------- 15.8/198.6 MB 108.8 MB/s eta 0:00:02
   ---- ---------------------------------- 21.3/198.6 MB 131.2 MB/s eta 0:00:02
   ----- --------------------------------- 26.4/198.6 MB 131.2 MB/s eta 0:00:02
   ------ -------------------------------- 31.6/198.6 MB 129.5 MB/s eta 0:00:02
   ------- ------------------------------- 36.6/198.6 MB 110.0 MB/s eta 0:00:02
   -------- ------------------------------ 41.8/198.6 MB 131.2 MB/s eta 0:00:02
   -------- ------------------------------- 44.1/198.6 MB 81.8 MB/s eta 0:00:02
   --------- ------------------------------ 47.8/198.6 MB 81.8 

  You can safely remove it manually.


In [10]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [11]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

AttributeError: module 'torch' has no attribute 'SymInt'

In [ ]:
##mock

In [31]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader([for uid in uids : objects[uid]], batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

SyntaxError: invalid syntax (3836733143.py, line 11)

In [1]:
# Original GAN Code (in PyTorch)
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
print(torch.__version__)

1.12.1


In [19]:

# Define the generator class
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # ... (architecture details)
#         self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            
            
        )

    def forward(self, x):
        return self.fc(x)

In [24]:

# Define the discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # ... (architecture details)
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
            

    def forward(self, x):
        return self.fc(x)

In [21]:
# Create generator and discriminator instances
generator = Generator()
print(generator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [26]:

discriminator = Discriminator()
print(discriminator)


Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [29]:
# # Initialize the ``BCELoss`` function
# criterion = nn.BCELoss()

# # Create batch of latent vectors that we will use to visualize
# #  the progression of the generator
# fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# # Establish convention for real and fake labels during training
# real_label = 1.
# fake_label = 0.

# # Setup Adam optimizers for both G and D
# optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
# optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.001)

In [ ]:
# Training loop
for epoch in range(1000):
    # ... (training details)

In [ ]:
# Mock data for testing
mock_data = torch.tensor([[0, 1, 0, 0, 1, 1, 1],
                          [1, 0, 1, 1, 0, 0, 1],
                          [0, 0, 1, 1, 1, 0, 0]])

In [ ]:
# Print mock data and generated results
print("Mock data:")
print(mock_data)
print("\nGenerated data:")
print(generator(mock_data).round())

In [ ]:
https://github.com/black0017/3D-GAN-pytorch/blob/master/models/GAN3D.py